In [0]:
from pyspark.sql import functions as F
import json
from pyspark.sql.functions import *
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit, monotonically_increasing_id
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, DateType
)

In [0]:
# Parameters extraction
Parameters = dbutils.widgets.get("Parameters")
Parameters = json.loads(Parameters)

ProcessInstanceId = 0
ProcessQueueId = 0
StageId = 0
TableName = ""

for p in Parameters:
    if p.get("TableName") == "Customers":
        ProcessInstanceId = int(p.get("ProcessInstanceId"))
        ProcessQueueId = int(p.get("ProcessQueueId"))
        StageId = int(p.get("StageId"))
        TableName = str(p.get("TableName"))

In [0]:
# Mark current table as InProgress
if StageId == 4:
    spark.sql(f"""
        update control.processqueue
        set ProcessStatus = 'InProgress',
            ProcessStartTime = current_timestamp()
        where StageId = {StageId}
            and ProcessInstanceId = {ProcessInstanceId}
            and ProcessQueueId = {ProcessQueueId}
            and TableName = '{TableName}';
    """)
else:
    raise Exception(f"Stage Id is not relavent to R2B-transformation for table: {TableName}")

In [0]:
status = False

try: 
    # -------------------------------------------------
    # Read Bronze Customers
    # -------------------------------------------------
    customers_df = spark.table("workspace.bronze.customers")

    # -------------------------------------------------
    # Clean & standardize
    # -------------------------------------------------
    dim_customer_df = (
        customers_df
        .select(
            col("CustomerId").cast(IntegerType()),
            col("CustomerName").cast(StringType()),
            col("Age").cast(IntegerType()),
            col("Gender").cast(StringType()),
            col("City").cast(StringType()),
            col("JoinDate").cast(DateType())
        )
        .dropDuplicates(["CustomerId"])
    )

    # -------------------------------------------------
    # Add surrogate key
    # -------------------------------------------------
    dim_customer_df = dim_customer_df.withColumn(
        "CustomerKey",
        monotonically_increasing_id()
    )

    # -------------------------------------------------
    # Define schema for Unknown Customer
    # -------------------------------------------------
    unknown_schema = StructType([
        StructField("CustomerId", IntegerType(), False),
        StructField("CustomerName", StringType(), False),
        StructField("Age", IntegerType(), False),
        StructField("Gender", StringType(), False),
        StructField("City", StringType(), False),
        StructField("JoinDate", DateType(), True),
        StructField("CustomerKey", IntegerType(), False)
    ])

    unknown_customer = spark.createDataFrame(
        [(-1, "Unknown", -1, "Unknown", "Unknown", None, -1)],
        schema=unknown_schema
    )

    # -------------------------------------------------
    # Union Unknown record
    # -------------------------------------------------
    dim_customer_df = dim_customer_df.unionByName(unknown_customer)

    # -------------------------------------------------
    # Final column order
    # -------------------------------------------------
    dim_customer_df = dim_customer_df.select(
        "CustomerKey",
        "CustomerId",
        "CustomerName",
        "Age",
        "Gender",
        "City",
        "JoinDate"
    )

    # -------------------------------------------------
    # Write to Gold
    # -------------------------------------------------
    (
        dim_customer_df
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("workspace.gold.dim_customer")
    )
    status = True


# SCD-2 Implementation : ############################
    # ---------------------------------------
    # Source
    # ---------------------------------------
    source_df = (
        spark.table("workspace.bronze.customers")
        .select(
            col("CustomerId").cast("string"),
            col("CustomerName"),
            col("Age"),
            col("Gender"),
            col("City")
        )
        .withColumn(
            "RecordHash",
            sha2(concat_ws("||", "CustomerName", "Age", "Gender", "City"), 256)
        )
        .withColumn("StartDate", current_timestamp())
        .withColumn("EndDate", lit(None).cast("timestamp"))
        .withColumn("IsCurrent", lit(True))
    )

    # ---------------------------------------
    # Target
    # ---------------------------------------
    target = DeltaTable.forName(
        spark,
        "workspace.gold.dim_customer"
    )

    # ---------------------------------------
    # SCD-2 MERGE
    # ---------------------------------------
    (
        target.alias("t")
        .merge(
            source_df.alias("s"),
            "t.CustomerId = s.CustomerId"
        )
        # Expire current record if changed
        .whenMatchedUpdate(
            condition="t.IsCurrent = true AND t.RecordHash <> s.RecordHash",
            set={
                "EndDate": "current_timestamp()",
                "IsCurrent": "false"
            }
        )
        # Insert new version
        .whenNotMatchedInsert(
            values={
                "CustomerId": "s.CustomerId",
                "CustomerName": "s.CustomerName",
                "Age": "s.Age",
                "Gender": "s.Gender",
                "City": "s.City",
                "StartDate": "s.StartDate",
                "EndDate": "s.EndDate",
                "IsCurrent": "s.IsCurrent",
                "RecordHash": "s.RecordHash"
            }
        )
        .execute()
    )
    status = True

except Exception as e:
    print(f"Error: {e}")
    status = False


In [0]:
# Mark file as Success/Failed

if status == True:
    spark.sql(f"""
        UPDATE control.processqueue
        SET
            ProcessStatus = 'Succeeded',
            ProcessEndTime = current_timestamp(),
            ProcessDuration = CAST(
                (unix_timestamp(current_timestamp()) - unix_timestamp(ProcessStartTime)) / 60
                AS BIGINT
            )
        WHERE
            StageId = {StageId}
            AND ProcessInstanceId = {ProcessInstanceId}
            AND ProcessQueueId = {ProcessQueueId}
            AND TableName = '{TableName}'
            """)
    print(f"{TableName} Marked as Successful")
elif status == False:
        spark.sql(f"""
        UPDATE control.processqueue
        SET
            ProcessStatus = 'Failed',
            ProcessEndTime = current_timestamp(),
            ProcessDuration = CAST(
                (unix_timestamp(current_timestamp()) - unix_timestamp(ProcessStartTime)) / 60
                AS BIGINT
            )
        WHERE
            StageId = {StageId}
            AND ProcessInstanceId = {ProcessInstanceId}
            AND ProcessQueueId = {ProcessQueueId}
            AND TableName = '{TableName}'
            """)
        print(f"{TableName} Marked as Failed")
        raise Exception(f"Hard failure: {TableName} Failure detected")